# 1. Anomaly detection algorithms

Los outliers en el problema de zonificación son aquellos clientes alejados del grupo grande de ubicaciones, usualmente se encuentran en zonas alejadas a la ciudad → poblaciones. Estos clientes se remueven de la zonificación pues no siguen la lógica de este proceso.

<img src="https://scikit-learn.org/0.20/_images/sphx_glr_plot_anomaly_comparison_001.png" width="400" height="400" />

Para nuestro caso de estudio los métodos One-Class SVM, Isolation Forest y Local Outlier Factor son factibles. Robust covariance no pues obliga a uqe los datos se distribuya como una elipse y eso depende de la ciudad.

In [ ]:
import time

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.datasets import make_moons, make_blobs
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

# DataSets
files = ['fontibon.csv', 'quito.csv', 'tlalnepantla.csv', 'cali.csv', 'bucaramanga.csv']
datasets = []

for i in files:
    df = pd.read_csv(i)
    df.dropna()
    df = df.to_dict(orient = 'index')
    datasets.append( np.array([ [df[j]['lng'], df[j]['lat']] for j in df ]) )

# Detection algorithmn with different gamma value
anomaly_algorithms = [
    ("One-Class SVM 0.1", svm.OneClassSVM(kernel="rbf",gamma=0.1)),
    ("Robust covariance", EllipticEnvelope()),
    ("Isolation Forest", IsolationForest(random_state=42)),
    ("Local Outlier Factor", LocalOutlierFactor(n_neighbors=35))
]

plt.figure(figsize=(len(anomaly_algorithms) * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1

for i_dataset, X in enumerate(datasets):
    for name, algorithm in anomaly_algorithms:
        
        plt.subplot(len(datasets), len(anomaly_algorithms), plot_num)
        
        if i_dataset == 0:
            plt.title(name, size=15)

        # fit the data and tag outliers
        if name == "Local Outlier Factor":
            y_pred = algorithm.fit_predict(X)
        else:
            y_pred = algorithm.fit(X).predict(X)
        
        colors = np.array(['#377eb8', '#ff7f00'])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[(y_pred + 1) // 2])
        plt.xticks(())
        plt.yticks(())
        plot_num += 1

plt.show()

In [ ]:
# Not plotting outliers

plt.figure(figsize=(len(anomaly_algorithms) * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1

for i_dataset, X in enumerate(datasets):
    for name, algorithm in anomaly_algorithms:
        
        plt.subplot(len(datasets), len(anomaly_algorithms), plot_num)
        
        if i_dataset == 0:
            plt.title(name, size=15)

        # fit the data and tag outliers
        if name == "Local Outlier Factor":
            y_pred = algorithm.fit_predict(X)
        else:
            y_pred = algorithm.fit(X).predict(X)
        
        newX = []
        for idx, k in enumerate(y_pred):
            if k == 1:
                newX.append(X[idx])
                
        newX = np.array(newX)
        
        colors = np.array(['#377eb8', '#ff7f00'])
        plt.scatter(newX[:, 0], newX[:, 1], s=10, color=colors[1])
        plt.xticks(())
        plt.yticks(())
        plot_num += 1

plt.show()

In [ ]:
# DataSets
files = ['fontibon.csv', 'quito.csv', 'tlalnepantla.csv', 'cali.csv', 'bucaramanga.csv']
datasets = []

for i in files:
    df = pd.read_csv(i)
    df.dropna()
    df = df.to_dict(orient = 'index')
    datasets.append( np.array([ [df[j]['lng'], df[j]['lat']] for j in df ]) )

# Detection algorithmn with different gamma value
anomaly_algorithms = [
    ("I. Forest 0.05", IsolationForest(contamination=0.05, random_state=42)),
    ("I. Forest 0.10", IsolationForest(contamination=0.10, random_state=42)),
    ("I. Forest 0.20", IsolationForest(contamination=0.20, random_state=42)),
    ("I. Forest 0.30", IsolationForest(contamination=0.30, random_state=42)),
    ("I. Forest 0.40", IsolationForest(contamination=0.40, random_state=42)),
    ("I. Forest 0.50", IsolationForest(contamination=0.50, random_state=42))
]

# Not plotting outliers
plt.figure(figsize=(len(anomaly_algorithms) * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1
for i_dataset, X in enumerate(datasets):
    for name, algorithm in anomaly_algorithms:
        
        plt.subplot(len(datasets), len(anomaly_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=15)

        # fit the data and tag outliers
        if name == "Local Outlier Factor":
            y_pred = algorithm.fit_predict(X)
        else:
            y_pred = algorithm.fit(X).predict(X)
        
        newX = []
        for idx, k in enumerate(y_pred):
            if k == 1:
                newX.append(X[idx])
                
        newX = np.array(newX)
        
        colors = np.array(['#377eb8', '#ff7f00'])
        plt.scatter(newX[:, 0], newX[:, 1], s=10, color=colors[1])
        plt.xticks(())
        plt.yticks(())
        plot_num += 1

plt.show()

# Clustering

Una vez se eliminan los datos atipicos de direcciones se sigue con el problema de zonificación o clusterización. Para este problema el equipo comercial define cuántas zonas quiere obtener, ya sea por el # de hunters que tiene en el momento, o por el # de hunters que se planean contratar.

Se tiene que tener en cuenta que aquellos outliers removidos en la fase anterior no se van a tener en cuenta en el nuevo ruteo ni van a contar en el número de zonas definidas por comercial.

In [ ]:
def objectiveFun(df: dict, n) -> int:
    F_OB = { i: 0 for i in range(n)}
    for i in df:
        F_OB[df[i]['zone']] += 1

    nMax = max([F_OB[i] for i in F_OB])
    nMin = min([F_OB[i] for i in F_OB])
    
    return nMax,nMin

In [ ]:
contaminationLS = np.arange(0.1,0.5,0.05) # Diferent outliers filtering by Isolation Forest
nClusters = 30

# Not plotting outliers
plt.figure(figsize=(22, 26))
plt.subplots_adjust(left=.02, right=.98, bottom=.02, top=.96, wspace=.05,hspace=.12)

plot_num = 1
for contamination in contaminationLS:
    start_time = time.time()
    # Test case - Bodega Fontibon
    df = pd.read_csv('fontibonCPS.csv')
    df.dropna()
    df = df.to_dict(orient = 'index')

    X = []
    IdK = {}
    count = 0
    for j in df:
        X.append([df[j]['lng'], df[j]['lat']])
        IdK[count] = j
        count += 1
    X = np.array(X)
    outliers_algh = IsolationForest(contamination = contamination) # IsolationForest Algorithm

    ''' Outliers remover '''
    y_pred = outliers_algh.fit(X).predict(X) # Fit algorithm in data
    for idx, k in enumerate(y_pred):
        if k == -1:
            df.pop(IdK[idx])

    # Clean Data
    X = []
    IdK = {}
    count = 0
    for j in df:
        X.append([df[j]['lng'], df[j]['lat']])
        IdK[count] = j
        count += 1
    X = np.array(X)

    ''' Zonification '''

    from sklearn import cluster, mixture
    from sklearn.preprocessing import StandardScaler

    clustering_alorithms = [
        ('ward' , cluster.AgglomerativeClustering(n_clusters=nClusters, linkage='ward')),
        ('average' , cluster.AgglomerativeClustering(n_clusters=nClusters, linkage='average')),
        ('complete' , cluster.AgglomerativeClustering(n_clusters=nClusters, linkage='complete')),
        ('single' , cluster.AgglomerativeClustering(n_clusters=nClusters, linkage='single')),
        ('kMeans' , cluster.KMeans(nClusters)),
        ('spectral nn' , cluster.SpectralClustering(n_clusters=nClusters, eigen_solver='arpack', affinity="nearest_neighbors")),
        ('spectal rbf' , cluster.SpectralClustering(n_clusters=nClusters, eigen_solver='arpack', affinity="rbf")),
        ('birch' , cluster.Birch(n_clusters=nClusters, threshold=0)),
        ('gaussian' , mixture.GaussianMixture(n_components=nClusters, covariance_type='full'))
    ]

    X = StandardScaler().fit_transform(X)

    for name, algorithm in clustering_alorithms:
        algorithm.fit(X)

        if hasattr(algorithm, 'labels_'):
            z_pred = algorithm.labels_.astype(int)
        else:
            z_pred = algorithm.predict(X)

        for idx, k in enumerate(z_pred):
            df[IdK[idx]]['zone'] = k

        plt.subplot( 8, len(clustering_alorithms), plot_num)
        plt.scatter( [df[i]['lng'] for i in df], [df[i]['lat'] for i in df], c = [df[i]['zone'] for i in df])
        plt.xticks(())
        plt.yticks(())
        plt.title( name + str(round(contamination,2)) + str(objectiveFun(df,30)))
        plot_num += 1